In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import os

import itertools

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. Dictionary

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [5]:
df_dict_main = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name=COMBINE_harmonizer.SHEET_MAIN)
df_dict_followup = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name=COMBINE_harmonizer.SHEET_FOLLOW_UP)


## 03. Mapping

In [6]:
_MAIN_ORDER_MAP = COMBINE_harmonizer.build_variable_order_map(data_dict_filename, sheet_name=COMBINE_harmonizer.SHEET_MAIN)

_FOLLOWUP_ORDER_MAP = COMBINE_harmonizer.build_variable_order_map(data_dict_filename, sheet_name=COMBINE_harmonizer.SHEET_FOLLOW_UP)

In [7]:
_MAIN_ORDER_MAP

{'_study': 0,
 'center': 1,
 'subjectID': 2,
 'uniqueID': 3,
 'MRI_ID': 4,
 'followupCenter': 5,
 'followupID': 6,
 'uniqueFollowupID': 7,
 '_flatten_index': 8,
 'siteID': 11,
 'birthDate': 176,
 'birthNumber': 13,
 'screenComment': 14,
 'coreTempLess32p5CGreaterEq2Hr_e': 15,
 'coreTempLess33p5CGreater1Hr_e': 16,
 'coreTempLess34CGreater1Hr_e': 17,
 'first6HrCoolByClinicalProtocol_e': 18,
 'chromosomalAbnormality_e': 19,
 'majorCongenitalAnomaly_e': 20,
 'birthWeightLessEq1800g_e': 21,
 'infantUnlikelySurvive_e': 22,
 'first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e': 23,
 'postnatalAgeLess6HrOrGreater24Hr_e': 24,
 'enrolledConflictingTrial_e': 25,
 'first60MinAnyBloodGasPHLessEq7_i': 26,
 'first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i': 27,
 'historyPerinatalEvent_i': 28,
 'at10MinApgarLessEq5OrVent_i': 29,
 'randomEligible': 30,
 'consentStatus': 31,
 'noConsentReason': 32,
 'noInStudyReason': 33,
 'random': 34,
 'noRandomReason': 35,
 'noRandomReasonText': 36,
 '

In [8]:
_FOLLOWUP_ORDER_MAP

{'_study': 0,
 'center': 1,
 'subjectID': 2,
 'uniqueID': 3,
 'MRI_ID': 4,
 'followupCenter': 5,
 'followupID': 6,
 'uniqueFollowupID': 7,
 '_flatten_index': 8,
 'siteID': 1118,
 'birthDate': 1119,
 'visitDate': 1120,
 'birthNumber': 1121,
 'center_orig': 1125,
 'SESVisitDate': 1126,
 'SESBirthDate': 1127,
 'chronologicalAge_mo': 1128,
 'correctedAge_mo': 1129,
 'underStateSupervision': 1130,
 'primaryCaretaker': 1131,
 'otherCaretaker': 1132,
 'maritalStatusPrimaryCaretaker': 1133,
 'livingArrangementChild': 1134,
 'numberPeopleInChildHousehold': 1135,
 'otherContributeMoneyToChildHousehold': 1136,
 'educationPrimaryCaretaker': 1137,
 'educationOtherCaretaker': 1138,
 'workPrimaryCaretaker': 1139,
 'workOtherCaretaker': 1140,
 'inSchoolPrimaryCaretaker': 1141,
 'inSchoolOtherCaretaker': 1142,
 'totalIncomeChildHousehold': 1143,
 'medicalInsuranceChild': 1144,
 'primaryLanguageChild': 1145,
 'primaryLanguageChildOtherText': 1146,
 'isSecondaryLanguageChild': 1147,
 'secondaryLanguageCh

## 04. Filenames and Merge

In [9]:
out_dir = cfg.config['out_dir']
_DIRS = {
    'LH': f'{out_dir}/out-LH-normalized',
    'OC': f'{out_dir}/out-OC-normalized',
}

_FILENAMES = [each['name'] for each in COMBINE_harmonizer.FILENAME_INFOS]

_FILENAME_INFO_MAP = {each['name']: each for each in COMBINE_harmonizer.FILENAME_INFOS}

_OUT_DIR = f'{out_dir}/out-merged-normalized'
os.makedirs(_OUT_DIR, exist_ok=True)

In [10]:
def _get_order(x, order_map):
    if x not in order_map:
        print(f'[WARN] not in order_map: {x}')
        return COMBINE_harmonizer.MAX_INT

    return order_map[x]


def _column_and_orig(column, columns_with_orig):
    ret = [column]

    # XXX remove .orig in merge
    # if column + '.orig' in columns_with_orig:
    #     ret += [column + '.orig']

    return ret


def _reorder_columns(df, filename):
    columns_without_orig = list(filter(lambda x: not x.endswith('.orig'), df.columns))
    columns_with_orig = list(filter(lambda x: x.endswith('.orig'), df.columns))

    each_filename_info = _FILENAME_INFO_MAP[filename]
    order_map = _MAIN_ORDER_MAP
    if each_filename_info['data_dict'] == COMBINE_harmonizer.SHEET_FOLLOW_UP:
        order_map = _FOLLOWUP_ORDER_MAP

    columns_without_orig.sort(key=lambda x: _get_order(x, order_map))
    columns_with_orig.sort(key=lambda x: _get_order(x[:-5], order_map))

    reserved_columns = list(filter(lambda x: x in columns_without_orig, COMBINE_harmonizer.RESERVED_COLUMNS))
    other_columns = list(filter(lambda x: x not in COMBINE_harmonizer.RESERVED_COLUMNS, columns_without_orig))

    columns_and_origs = list(itertools.chain.from_iterable([_column_and_orig(each, columns_with_orig) for each in other_columns]))

    columns = reserved_columns + columns_and_origs

    return df[columns]

In [11]:
for idx, filename in enumerate(_FILENAMES):
    df = None
    for study, each_dir in _DIRS.items():
        each_filename = os.sep.join([each_dir, filename])
        if not os.path.exists(each_filename):
            continue

        each_df = pd.read_csv(each_filename, dtype='O')
        each_df['_study'] = study

        if df is None:
            df = each_df
        else:
            df = pd.concat([df, each_df])

    if df is None:
        print(f'[WARN] ({idx}/{len(_FILENAMES)}) not df: filename: {filename}')
        continue

    df = _reorder_columns(df, filename)

    out_filename = os.sep.join([_OUT_DIR, filename])

    df.to_csv(out_filename, index=False)

    print(f'[INFO] ({idx}/{len(_FILENAMES)}) done: filename: {filename} out_filename: {out_filename}')

[INFO] (0/73) done: filename: 00-02-screening.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/00-02-screening.csv


[INFO] (1/73) done: filename: 00-12-neuro-exam.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/00-12-neuro-exam.csv


[INFO] (2/73) done: filename: 01-02-screening.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-02-screening.csv


[INFO] (3/73) done: filename: 01-03-maternal-demographics.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-03-maternal-demographics.csv
[INFO] (4/73) done: filename: 01-04-pregnancy-history.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-04-pregnancy-history.csv


[INFO] (5/73) done: filename: 01-05-labor-delivery.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-05-labor-delivery.csv
[WARN] (6/73) not df: filename: 01-05_1-pse.csv
[WARN] (7/73) not df: filename: 01-05_2-emergency-csection.csv


...

[INFO] (66/73) done: filename: 20-05-gmfcs.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-05-gmfcs.csv


[INFO] (67/73) done: filename: 20-06-status.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-06-status.csv


[INFO] (68/73) done: filename: 20-07-readmission.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-07-readmission.csv


[INFO] (69/73) done: filename: 20-08-lost.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-08-lost.csv
[INFO] (70/73) done: filename: 20-09-secondary.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-09-secondary.csv


[INFO] (71/73) done: filename: 20-10-outcome.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-10-outcome.csv
[WARN] (72/73) not df: filename: 20-10_1-disability-level-death.csv
